In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Pragyasharava\\Desktop\\leukemia-detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pragyasharva/leukemia-detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pragyasharva"
os.environ["MLFLOW_TRACKING_PASSWORD"]="7072512c2294bd7b378427f2ba8b8f393315356b"

In [6]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model",
            training_data="artifacts/data_ingestion/Blood cell Cancer [ALL]",
            mlflow_uri="https://dagshub.com/pragyasharva/leukemia-detection.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="efficientnet_b3_model")
            else:
                mlflow.tensorflow.log_model(self.model, "model")        

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-23 19:14:23,547: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 19:14:23,552: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 19:14:23,553: INFO: common: created directory at: artifacts]
Found 970 images belonging to 4 classes.
61/61 [==============================] - 48s 732ms/step - loss: 0.2715 - accuracy: 0.9320
[2025-02-23 19:15:28,070: INFO: common: json file saved at: scores.json]


2025/02/23 19:15:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\PRAGYA~1\AppData\Local\Temp\tmpshzzojuk\model\data\model\assets
[2025-02-23 19:16:04,358: INFO: builder_impl: Assets written to: C:\Users\PRAGYA~1\AppData\Local\Temp\tmpshzzojuk\model\data\model\assets]


2025/02/23 19:16:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'efficientnet_b3_model'.
2025/02/23 19:17:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: efficientnet_b3_model, version 1
Created version '1' of model 'efficientnet_b3_model'.
2025/02/23 19:17:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run magnificent-stag-347 at: https://dagshub.com/pragyasharva/leukemia-detection.mlflow/#/experiments/0/runs/8bc2d40c17604af5a712781610c94428.
2025/02/23 19:17:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pragyasharva/leukemia-detection.mlflow/#/experiments/0.
